In [11]:
# ================== НАСТРОЙКИ ==================

TELEGRAM_TOKEN = "7780754536:AAF0vZ_1kwgErTlK5Ah44YzByEaQqVgyEls"
GIGACHAT_AUTH_KEY = "MDE5YjlmZTgtMjQ0Mi03OTgzLTk5YzQtNTcwNjI5OTFmMTEyOjNmNjI0MjBhLTEwOGYtNGY3NC04NjM2LThmMzk1ODk3Njg0OQ=="

# ================== ИМПОРТЫ ==================

!pip install -q python-telegram-bot==20.7 requests nest-asyncio

import requests
import asyncio
import base64
import time
import nest_asyncio
from telegram import (
    Update,
    InlineKeyboardButton,
    InlineKeyboardMarkup
)
from telegram.ext import (
    ApplicationBuilder,
    CommandHandler,
    MessageHandler,
    CallbackQueryHandler,
    ContextTypes,
    filters
)

nest_asyncio.apply()

# ================== GIGACHAT API ==================

class GigaChatAPI:
    def __init__(self, auth_key: str):
        self.auth_key = auth_key
        self.access_token = None
        self.token_expires = 0
        self.last_request_time = 0
        self.request_delay = 3.0  # Увеличил задержку до 3 секунд
        self.rate_limit_count = 0

    def _get_access_token(self) -> str:
        """Получение OAuth токена для GigaChat"""
        # Проверяем, нужно ли получать новый токен
        if self.access_token and time.time() < self.token_expires:
            return self.access_token

        url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

        headers = {
            "Authorization": f"Basic {self.auth_key}",
            "Content-Type": "application/x-www-form-urlencoded",
            "Accept": "application/json",
            "RqUID": str(int(time.time() * 1000))
        }

        data = "scope=GIGACHAT_API_PERS"

        try:
            # Временное отключение предупреждений о сертификатах
            import urllib3
            urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

            response = requests.post(
                url,
                headers=headers,
                data=data,
                verify=False,
                timeout=10
            )

            print(f"📡 Статус ответа авторизации: {response.status_code}")

            if response.status_code == 200:
                result = response.json()
                self.access_token = result.get("access_token")
                # Токен действует 30 минут (1800 секунд), ставим запас 5 минут
                self.token_expires = time.time() + 1500
                print(f"✅ Токен получен успешно, действителен до: {time.ctime(self.token_expires)}")
                return self.access_token
            else:
                print(f"❌ Ошибка авторизации GigaChat: {response.status_code}")
                print(f"Ответ сервера: {response.text}")
                return None

        except Exception as e:
            print(f"❌ Ошибка получения токена GigaChat: {e}")
            return None

    def generate_text(self, prompt: str, max_retries: int = 2) -> str:
        """Генерация текста через GigaChat API с повторными попытками"""
        for attempt in range(max_retries + 1):
            try:
                # Задержка для избежания лимита запросов
                current_time = time.time()
                time_since_last_request = current_time - self.last_request_time
                if time_since_last_request < self.request_delay:
                    sleep_time = self.request_delay - time_since_last_request
                    print(f"⏳ Задержка {sleep_time:.1f} секунд...")
                    time.sleep(sleep_time)

                self.last_request_time = time.time()

                # Получаем токен если его нет или истек
                if not self.access_token or time.time() >= self.token_expires:
                    print("🔄 Получаю новый токен...")
                    token = self._get_access_token()
                    if not token:
                        return "❌ Не удалось получить токен доступа к GigaChat API. Проверьте ключ."

                url = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"

                headers = {
                    "Authorization": f"Bearer {self.access_token}",
                    "Content-Type": "application/json",
                    "Accept": "application/json"
                }

                # Упрощенный промпт для теста
                payload = {
                    "model": "GigaChat",
                    "messages": [
                        {"role": "user", "content": prompt[:500]}  # Ограничиваем промпт
                    ],
                    "temperature": 0.7,
                    "max_tokens": 800,  # Еще меньше токенов для теста
                    "stream": False
                }

                print(f"📤 Отправляю запрос к GigaChat (попытка {attempt + 1})...")

                response = requests.post(
                    url,
                    headers=headers,
                    json=payload,
                    verify=False,
                    timeout=45  # Увеличил таймаут
                )

                print(f"📥 Статус запроса к GigaChat: {response.status_code}")

                if response.status_code == 200:
                    result = response.json()
                    if "choices" in result and len(result["choices"]) > 0:
                        content = result["choices"][0]["message"]["content"]
                        print(f"✅ Успешно получен ответ ({len(content)} символов)")
                        # Сбрасываем счетчик лимитов при успехе
                        if self.rate_limit_count > 0:
                            self.rate_limit_count -= 1
                        return content
                    else:
                        return "Ошибка: неверный формат ответа"

                elif response.status_code == 401:
                    # Токен недействителен, обновляем
                    print("🔄 Токен устарел, обновляю...")
                    self.access_token = None
                    self.token_expires = 0
                    continue  # Повторяем попытку с новым токеном

                elif response.status_code == 429:
                    # Превышен лимит запросов
                    self.rate_limit_count += 1
                    self.request_delay = min(10.0, 3.0 + self.rate_limit_count * 2)  # Макс 10 сек
                    wait_time = 5 + self.rate_limit_count * 5  # 5, 10, 15 сек...

                    if attempt < max_retries:
                        print(f"⚠️ Лимит запросов. Жду {wait_time} секунд перед повторной попыткой...")
                        time.sleep(wait_time)
                        continue
                    else:
                        print(f"⚠️ Превышен лимит запросов после {max_retries + 1} попыток.")
                        return f"⚠️ Превышен лимит запросов к GigaChat. Попробуйте позже (через {wait_time} секунд)."

                elif response.status_code == 403:
                    return "❌ Доступ запрещен. Проверьте права доступа API ключа."

                elif response.status_code == 500:
                    if attempt < max_retries:
                        print(f"🔄 Ошибка сервера 500. Жду 3 секунды перед повторной попыткой...")
                        time.sleep(3)
                        continue
                    return "❌ Внутренняя ошибка сервера GigaChat."

                else:
                    print(f"📄 Ответ ошибки: {response.text[:200]}")
                    return f"❌ Ошибка GigaChat API ({response.status_code})"

            except requests.exceptions.Timeout:
                if attempt < max_retries:
                    print(f"⏱️ Таймаут. Жду 5 секунд перед повторной попыткой...")
                    time.sleep(5)
                    continue
                return "⏱️ Таймаут запроса к GigaChat API"

            except Exception as e:
                if attempt < max_retries:
                    print(f"🔄 Ошибка: {str(e)}. Жду 3 секунды перед повторной попыткой...")
                    time.sleep(3)
                    continue
                return f"❌ Ошибка: {str(e)}"

        return "❌ Не удалось выполнить запрос после нескольких попыток."

# Инициализируем GigaChat
gigachat = GigaChatAPI(GIGACHAT_AUTH_KEY)

# ================== ПРОМПТЫ ДЛЯ GIGACHAT ==================

def get_instagram_prompt(user_text: str) -> str:
    """Создание поста для Instagram"""
    short_text = user_text[:300]  # Берем только начало текста
    return f"""Создай короткий Instagram пост на основе: "{short_text}"

Требования:
1. Заголовок с 1-2 эмодзи
2. 2 коротких абзаца основного текста
3. Вопрос для читателей
4. 3-5 хэштегов
5. Максимум 500 символов

Сделай текст живым и интересным!"""

def get_shorten_prompt(user_text: str) -> str:
    """Сокращение текста"""
    short_text = user_text[:400]  # Берем только начало текста
    return f"""Сократи этот текст до 150-300 символов, сохранив главную мысль: "{short_text}"

Удали воду, оставь только самое важное. Будь кратким."""

def get_expand_prompt(user_text: str) -> str:
    """Расширение текста"""
    short_text = user_text[:200]  # Берем только начало текста
    return f"""Расширь этот текст до 300-500 символов: "{short_text}"

Добавь немного деталей и примеров, но будь кратким."""

# ================== TELEGRAM БОТ ==================

USER_TEXT = {}
USER_STATE = {}  # Для отслеживания состояния пользователя

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Команда /start"""
    await update.message.reply_text(
        "🤖 *GigaChat Content Bot*\n\n"
        "Я создаю контент с помощью нейросети GigaChat.\n\n"
        "📱 *Instagram Post* — готовый пост\n"
        "✂️ *Shorten Text* — сокращенная версия\n"
        "📈 *Expand Text* — подробный разбор\n\n"
        "⚠️ *Ограничение:* 1 запрос в 10 секунд\n"
        "Отправьте мне текст (5-1000 символов)!",
        parse_mode="Markdown"
    )

async def gigachat_info(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Информация о GigaChat"""
    await update.message.reply_text(
        "ℹ️ *О GigaChat:*\n\n"
        "• Русскоязычная нейросеть от Сбера\n"
        "• API имеет лимиты по запросам\n"
        "• Для теста используйте короткие тексты\n"
        "• При ошибке 429 — подождите 30 секунд\n\n"
        "✨ Используется GigaChat API",
        parse_mode="Markdown"
    )

async def handle_text(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработка текста от пользователя"""
    user_id = update.effective_user.id
    text = update.message.text.strip()

    # Проверяем ограничения
    if len(text) < 5:
        await update.message.reply_text("📝 Отправьте текст подлиннее (минимум 5 символов)")
        return

    if len(text) > 1000:
        await update.message.reply_text("⚠️ Для теста используйте текст до 1000 символов")
        text = text[:1000]  # Обрезаем текст

    # Проверяем частоту запросов
    current_time = time.time()
    if user_id in USER_STATE:
        time_since_last = current_time - USER_STATE[user_id].get('last_request', 0)
        if time_since_last < 10:  # 10 секунд между запросами
            wait_time = 10 - int(time_since_last)
            await update.message.reply_text(
                f"⏳ Подождите {wait_time} секунд перед следующим запросом"
            )
            return

    USER_STATE[user_id] = {'last_request': current_time}
    USER_TEXT[user_id] = text

    # Простая клавиатура с одним вариантом для теста
    keyboard = [
        [InlineKeyboardButton("📱 Instagram пост", callback_data="insta")],
        [InlineKeyboardButton("✂️ Сократить текст", callback_data="short")],
        [InlineKeyboardButton("📈 Расширить текст", callback_data="expand")]
    ]

    await update.message.reply_text(
        f"✅ *Текст получен!*\n\n"
        f"📊 Длина: {len(text)} символов\n"
        f"🧠 Выберите один вариант для теста:\n"
        f"⚠️ Работаю медленно из-за лимитов API",
        reply_markup=InlineKeyboardMarkup(keyboard),
        parse_mode="Markdown"
    )

async def handle_button(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработка нажатий кнопок"""
    query = update.callback_query
    await query.answer()

    user_id = query.from_user.id
    user_text = USER_TEXT.get(user_id)

    if not user_text:
        await query.edit_message_text("❌ Текст не найден. Отправьте текст снова.")
        return

    # Определяем действие
    if query.data == "insta":
        action_name = "Instagram пост"
        prompt_function = get_instagram_prompt
        result_title = "📱 *Instagram Post*\n\n"
    elif query.data == "short":
        action_name = "Сокращенный текст"
        prompt_function = get_shorten_prompt
        result_title = "✂️ *Shortened Text*\n\n"
    elif query.data == "expand":
        action_name = "Расширенный текст"
        prompt_function = get_expand_prompt
        result_title = "📈 *Expanded Text*\n\n"
    else:
        await query.edit_message_text("❌ Неизвестная команда")
        return

    # Обновляем сообщение о статусе
    await query.edit_message_text(
        f"🔄 *Создаю {action_name}...*\n\n"
        f"Использую GigaChat AI\n"
        f"⚠️ Может занять 15-30 секунд\n"
        f"⏳ Ожидайте...",
        parse_mode="Markdown"
    )

    try:
        # Генерируем промпт и получаем ответ от GigaChat
        prompt = prompt_function(user_text)
        result = gigachat.generate_text(prompt)

        # Проверяем результат
        if result.startswith("❌") or result.startswith("⚠️") or "Ошибка" in result or "лимит" in result:
            # Если ошибка лимита, предлагаем подождать
            if "лимит" in result.lower() or "429" in result:
                error_msg = (
                    "⚠️ *Превышен лимит запросов GigaChat*\n\n"
                    "GigaChat API имеет строгие лимиты:\n"
                    "• Подождите 30-60 секунд\n"
                    "• Используйте более короткие тексты\n"
                    "• Попробуйте позже\n\n"
                    "🔄 Отправьте текст снова через минуту"
                )
            else:
                error_msg = f"❌ *Ошибка:* {result}\n\nПопробуйте другой вариант или подождите."

            await query.edit_message_text(error_msg, parse_mode="Markdown")
            return

        # Успешный результат
        final_message = f"{result_title}{result}\n\n"
        final_message += f"---\n📊 *Статистика:*\n"
        final_message += f"• Исходный текст: {len(user_text)} символов\n"
        final_message += f"• Результат: {len(result)} символов\n"

        if len(user_text) > 0 and len(result) > 0:
            change_percent = (len(result) / len(user_text)) * 100
            final_message += f"• Изменение: {change_percent:.1f}% от оригинала\n"

        final_message += f"\n🔄 Отправьте новый текст для продолжения\n"
        final_message += f"⏳ Следующий запрос через 10 секунд"

        # Отправляем результат
        if len(final_message) > 4096:
            # Разбиваем на части
            parts = [final_message[i:i+4000] for i in range(0, len(final_message), 4000)]
            await query.edit_message_text(parts[0], parse_mode="Markdown")
            for part in parts[1:]:
                await query.message.reply_text(part, parse_mode="Markdown")
        else:
            await query.edit_message_text(final_message, parse_mode="Markdown")

    except Exception as e:
        error_msg = f"❌ *Ошибка при обработке:*\n{str(e)[:200]}\n\nПопробуйте снова через минуту."
        await query.edit_message_text(error_msg, parse_mode="Markdown")

async def error_handler(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Глобальный обработчик ошибок"""
    print(f"Произошла ошибка: {context.error}")
    try:
        if update and update.effective_message:
            await update.effective_message.reply_text(
                "❌ Произошла ошибка при обработке запроса.\n"
                "Попробуйте еще раз или отправьте текст заново."
            )
    except:
        pass

# ================== ЗАПУСК БОТА ==================

def main():
    print("=" * 60)
    print("🤖 GIGACHAT CONTENT BOT (УПРОЩЕННАЯ ВЕРСИЯ)")
    print("=" * 60)

    print(f"\n🔑 Проверяю ключ GigaChat...")
    print(f"✅ Ключ настроен: {GIGACHAT_AUTH_KEY[:30]}...")

    # Проверяем подключение к GigaChat
    print("\n🔍 Тестируем подключение к GigaChat API...")

    # Пытаемся получить токен
    token = gigachat._get_access_token()
    if not token:
        print("❌ Не удалось получить токен GigaChat.")
        print("\n🔧 Возможные причины:")
        print("1. Неверный или просроченный ключ")
        print("2. API не активирован на developers.sber.ru")
        print("3. Проблемы с сетью")
        return

    print("✅ Токен получен успешно!")

    # Делаем ОДИН тестовый запрос
    print("\n🧪 Делаю ОДИН тестовый запрос к GigaChat...")
    test_result = gigachat.generate_text("Привет! Ответь одним словом.")

    if test_result.startswith("❌") or test_result.startswith("⚠️") or "лимит" in test_result:
        print(f"⚠️ Тестовый запрос не удался: {test_result[:100]}")
        print("\n💡 Рекомендации:")
        print("1. Подождите 1-2 минуты")
        print("2. GigaChat имеет строгие лимиты")
        print("3. Используйте короткие тексты (до 500 символов)")
        print("4. Делайте запросы раз в 15-20 секунд")
    else:
        print(f"✅ GigaChat API работает! Ответ: {test_result[:80]}...")
        print("\n⚠️ Внимание: GigaChat имеет строгие лимиты запросов!")
        print("• Делайте запросы не чаще 1 раза в 15 секунд")
        print("• Используйте короткие тексты")
        print("• При ошибке 429 — ждите 30-60 секунд")

    print("\n🚀 Запуск Telegram бота...")

    # Создаем приложение
    app = ApplicationBuilder()\
        .token(TELEGRAM_TOKEN)\
        .concurrent_updates(True)\
        .pool_timeout(30)\
        .build()

    # Регистрируем обработчики
    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("gigachat", gigachat_info))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_text))
    app.add_handler(CallbackQueryHandler(handle_button))
    app.add_error_handler(error_handler)

    print("✅ Бот запущен успешно!")
    print("📱 Имя бота: GigaChat Content Bot")
    print("⚡ Используется: GigaChat API")
    print("⚠️ Ограничения: 1 запрос / 15 секунд")
    print("⏹️ Для остановки: Ctrl+C")
    print("-" * 60)

    try:
        app.run_polling(
            drop_pending_updates=True,
            timeout=30,
            allowed_updates=Update.ALL_TYPES,
            close_loop=False
        )
    except KeyboardInterrupt:
        print("\n🛑 Бот остановлен")
    except Exception as e:
        print(f"💥 Критическая ошибка: {e}")

if __name__ == "__main__":
    main()

🤖 GIGACHAT CONTENT BOT (УПРОЩЕННАЯ ВЕРСИЯ)

🔑 Проверяю ключ GigaChat...
✅ Ключ настроен: MDE5YjlmZTgtMjQ0Mi03OTgzLTk5Yz...

🔍 Тестируем подключение к GigaChat API...
📡 Статус ответа авторизации: 400
❌ Ошибка авторизации GigaChat: 400
Ответ сервера: 
❌ Не удалось получить токен GigaChat.

🔧 Возможные причины:
1. Неверный или просроченный ключ
2. API не активирован на developers.sber.ru
3. Проблемы с сетью
